<a href="https://colab.research.google.com/github/hayamaster/Deep-learning/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pylab
import numpy as np

#Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 50
batch_size = 32
sample_dir = 'samples'
save_dir = 'save'

#Create a directory if not exists
if not os.path.exists(sample_dir):
  os.makedirs(sample_dir)

if not os.path.exists(save_dir):
  os.makedirs(save_dir)


#image processing
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize([0.5],[0.5])
])


#MNIST dataset
mnist = torchvision.datasets.MNIST(root='./data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

#Data loader 
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size,
                                          shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
#Generator
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size,hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

#Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)

#Device setting
D = D.cuda()
G = G.cuda()

#Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr = 0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr = 0.0002)

def denorm(x):
  out = (x+1)/2
  return out.clamp(0,1)

def reset_grad():
  d_optimizer.zero_grad()
  g_optimizer.zero_grad()


#Statistics to be saved
d_losses = np.zeros(num_epochs)
g_losses = np.zeros(num_epochs)
real_scores = np.zeros(num_epochs)
fake_scores = np.zeros(num_epochs)

In [ ]:
#Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
  for i, (images,_) in enumerate(data_loader):
    images = images.view(batch_size, -1).cuda()
    images = Variable(images)

    #Create the labels which are later used as input for the BCD loss 
    real_labels = torch.ones(batch_size, 1).cuda()
    real_labels = Variable(real_labels)
    fake_labels = torch.zeros(batch_size, 1).cuda()
    fake_labels = Variable(fake_labels)

    outputs = D(images)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs

    z = torch.randn(batch_size, latent_size).cuda()
    z = Variable(z)
    fake_images = G(z)
    outputs = D(fake_images).detach()
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs

    d_loss = d_loss_real + d_loss_fake

    #reset_grad
    d_optimizer.zero_grad()
    d_loss.backward()
    d_optimizer.step()

    z = torch.randn(batch_size, latent_size).cuda()
    z = Variable(z)
    fake_images = G(z)
    outputs = D(fake_images)

    g_loss = criterion(outputs, real_labels)

    #reset_grad()
    g_optimizer.zero_grad()
    g_loss.backward()
    g_optimizer.step()

    d_losses[epoch] = d_losses[epoch]*(i/(i+1.)) + d_loss.item()*(1./(i+1.))
    g_losses[epoch] = g_losses[epoch]*(i/(i+1.)) + g_loss.item()*(1./(i+1.))
    real_scores[epoch] = real_scores[epoch]*(i/(i+1.)) + real_score.mean().item()*(1./(i+1.))
    fake_scores[epoch] = fake_scores[epoch]*(i/(i+1.)) + fake_score.mean().item()*(1./(i+1.))

    if(i + 1)%200 ==0:
      print('Epoch[{}/{}], Step[{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'
            .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(),
                    real_score.mean().item(), fake_score.mean().item()))
  

  #Save real images
    if(epoch + 1) == 1:
      images = images.view(images.size(0), 1, 28, 28)
      save_image(denorm(images.data), os.path.join(sample_dir, 'real_images.png'))

    #Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images.data), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch + 1)))

    #Save and plot Statistics
    np.save(os.path.join(save_dir, 'd_losses.npy'), d_losses)
    np.save(os.path.join(save_dir, 'g_losses.npy'), g_losses)
    np.save(os.path.join(save_dir, 'fake_scores.npy'), fake_scores)
    np.save(os.path.join(save_dir, 'real_scores.npy'), real_scores)

    plt.figure()
    pylab.xlim(0, num_epochs+1)
    pylab.ylim(0,1)
    plt.plot(range(1,num_epochs + 1), fake_scores, label='fake score')
    plt.plot(range(1, num_epochs + 1), real_scores, label='real score')
    plt.legend()
    plt.savefig(os.path.join(save_dir, 'accuaracy.pdf'))
    plt.close()

    #Save model at checkpoints
    if(epoch + 1)%50 == 0:
      torch.save(G.state_dict(), os.path.join(save_dir, 'G--{}.ckpt'.format(epoch + 1)))
      torch.save(D.state_dict(), os.path.join(save_dir, 'D--{}.ckpt'.format(epoch + 1)))

#Save the model checkpoints
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch[0/50], Step[200/1875], d_loss: 10.2857, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[0/50], Step[400/1875], d_loss: 12.5129, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[0/50], Step[600/1875], d_loss: 13.8771, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[0/50], Step[800/1875], d_loss: 14.0782, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[0/50], Step[1000/1875], d_loss: 14.9183, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[0/50], Step[1200/1875], d_loss: 20.2180, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[0/50], Step[1400/1875], d_loss: 15.0989, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[0/50], Step[1600/1875], d_loss: 28.4367, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[0/50], Step[1800/1875], d_loss: 23.2223, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[1/50], Step[200/1875], d_loss: 36.4544, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[1/50], Step[400/1875], d_loss: 41.6784, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch[1/50], Step[600/1875]